In [ ]:
import os
import wfdb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

class Inception1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Inception1D, self).__init__()
        self.branch1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)

        self.branch2 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=1),
            nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=1),
            nn.Conv1d(out_channels, out_channels, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
            nn.Conv1d(in_channels, out_channels, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        outputs = torch.cat([branch1, branch2, branch3, branch4], dim=1)
        return outputs

class InceptionNet1D(nn.Module):
    def __init__(self, num_classes=5):
        super(InceptionNet1D, self).__init__()
        self.initial_conv = nn.Conv1d(1, 32, kernel_size=3, stride=2, padding=1)

        self.inception1 = Inception1D(32, 32)
        self.inception2 = Inception1D(128, 64)
        self.inception3 = Inception1D(256, 128)
        self.inception4 = Inception1D(512, 256)

        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = F.relu(self.initial_conv(x))
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.inception3(x)
        x = self.inception4(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

dummy_input = torch.randn(10, 1, 187)  # (batch_size, num_channels, seq_length)
model = InceptionNet1D()
output = model(dummy_input)
print(output.shape)  # (batch_size, num_classes)

train_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_train.csv', header=None)
test_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_test.csv', header=None)

X_train = train_df.iloc[:, :187].values
y_train = train_df.iloc[:, 187].values
X_test = test_df.iloc[:, :187].values
y_test = test_df.iloc[:, 187].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 1024

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

from tqdm import tqdm
lr = 0.001
num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = InceptionNet1D().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)  # Ajout d'une dimension pour le canal

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss/(i+1))
            pbar.update(1)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    with tqdm(total=len(test_loader), desc='Evaluating', unit='batch') as pbar:
        for inputs, labels in test_loader:
            inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            pbar.update(1)

print(f"Accuracy: {100 * correct / total}%")


torch.Size([10, 5])


Epoch 1/10:   0%|          | 0/86 [00:00<?, ?batch/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 32.55 GiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Epoch 1/10: 100%|██████████| 86/86 [00:28<00:00,  3.00batch/s, loss=0.642]


Epoch 1/10, Loss: 0.6415617861026941


Epoch 2/10: 100%|██████████| 86/86 [00:28<00:00,  2.98batch/s, loss=0.497]


Epoch 2/10, Loss: 0.4971895037695419


...

Epoch 9/10, Loss: 0.15937413179943727


Epoch 10/10: 100%|██████████| 86/86 [00:29<00:00,  2.91batch/s, loss=0.151]


Epoch 10/10, Loss: 0.15141315680257109


Evaluating: 100%|██████████| 22/22 [00:02<00:00,  8.92batch/s]

Accuracy: 96.10816736707473%


In [ ]:
class ECGDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.records = []
        self.labels = []
        self._load_data()

    def _load_data(self):
        for person_id in os.listdir(self.data_dir):
            person_path = os.path.join(self.data_dir, person_id)
            if os.path.isdir(person_path):
                for record_name in os.listdir(person_path):
                    if record_name.endswith('.hea'):
                        record_base = os.path.splitext(record_name)[0]
                        record_path = os.path.join(person_path, record_base)
                        record, fields = wfdb.rdsamp(record_path)
                        filtered_signal = record[:, 1]  # Column 1 is the filtered signal
                        self.records.append(filtered_signal)
                        self.labels.append(int(person_id.split('_')[1]))  # Assuming Person_01, Person_02, etc.

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        sample = self.records[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label

# Data transformation
def to_tensor(sample):
    return torch.tensor(sample, dtype=torch.float32)

# Directory containing the ECG data
data_dir = './datasets/ecg_id'

# Create the dataset
ecg_dataset = ECGDataset(data_dir, transform=to_tensor)

# Create the data loader
data_loader = DataLoader(ecg_dataset, batch_size=32, shuffle=False)

In [ ]:
def extract_features(model, data_loader):
    features = []
    labels_list = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.unsqueeze(1).to(device)
            outputs = model(inputs)

            features.append(outputs)
            labels_list.append(labels)
    features = torch.cat(features)
    labels_list = torch.cat(labels_list)
    return features, labels_list

test_features, test_labels = extract_features(model, data_loader)
from sklearn.metrics.pairwise import cosine_similarity

distances = cosine_similarity(test_features.detach().cpu(), test_features.detach().cpu())
np.fill_diagonal(distances, float('-inf'))
def find_max_indices(arr, k):
    if k > len(arr):
        raise ValueError("k cannot be greater than the size of the array")
    idx = np.argpartition(arr, -k)[-k:]
    sorted_idx = np.argsort(arr[idx])[::-1]
    return idx[sorted_idx]

def compute_top_k_accuracy(distances, test_labels, k):
    correct = 0
    for id_, elem in enumerate(distances, start=0):
        indices = find_max_indices(elem, k)
        candidates = [test_labels[indices[i]].item() for i in range(len(indices))]
        if test_labels[id_].item() in candidates:
            correct += 1
    accuracy = correct / len(distances)
    return accuracy

# Compute top-k accuracy
real_top_accuracies = []
for k in range(1, 6):
    accuracy = compute_top_k_accuracy(distances, test_labels, k)
    real_top_accuracies.append(accuracy)
    print(f"Top-{k} : {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Top-1 : 0.3161
Top-2 : 0.4387
Top-3 : 0.5000
Top-4 : 0.5581
Top-5 : 0.5935
